In [1]:
# Thermostat UI2

In [2]:
import requests
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, VBox
import time
import logging
from utils import *
import uuid
import smartpynector as sp

import ipywidgets as widgets
from IPython.display import display
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

In [5]:
label_actual = widgets.Label(value='', orientation='vertical')
progress = widgets.IntProgress(value=0, min=0, max=50, step=1, description='', bar_style='info', orientation='vertical')
# desired_temp_slider = IntSlider(value=0, min=0, max=50, orientation="vertical")
# label_desired = widgets.Label(value='', orientation='vertical')
# submit_button = Button(description="Submit", button_style="danger")

# desired_temp_slider.observe(update_desired_temp, names="value")
# submit_button.on_click(submit_changes)


In [6]:
def update_desired_temp(change):
    label_desired.value = f"{ change.new } ℃"

In [7]:
def handle_react_measurements(bindings):
    global temp_label_value
    for binding in bindings:
        progress.value = binding['temp']
        label_actual.value = f"{ binding['temp'] } ℃"
    return []

def handle_answer_measurements(query_bindings):
    global measurements
    # print(f'Answer query bindings: {query_bindings}')
    
    return match_bindings(query_bindings, measurements)

In [8]:
def start_ui_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    react_measurements_ki = register_react_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )
    
    ask_measurements_ki = register_ask_knowledge_interaction(
        GRAPH_PATTERN,
        "ask-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )


    start_handle_loop(
        {
            # answer_measurements_ki: handle_answer_measurements,
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )


In [9]:
def button_click():
    pass

In [10]:
display(HBox([label_actual,progress]))
start_ui_kb(
    "http://example.org/ui",
    "UI",
    "UI for measurement",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered UI
INFO:utils:received issued knowledge interaction id: http://example.org/ui/interaction/react-measurements
INFO:utils:received issued knowledge interaction id: http://example.org/ui/interaction/ask-measurements


KeyboardInterrupt: 